# Assignment 2 - Elementary Probability and Information Theory 
# Boise State University NLP - Dr. Kennington

### Instructions and Hints:

* This notebook loads some data into a `pandas` dataframe, then does a small amount of preprocessing. Make sure your data can load by stepping through all of the cells up until question 1. 
* Most of the questions require you to write some code. In many cases, you will write some kind of probability function like we did in class using the data. 
* Some of the questions only require you to write answers, so be sure to change the cell type to markdown or raw text
* Don't worry about normalizing the text this time (e.g., lowercase, etc.). Just focus on probabilies. 
* Most questions can be answered in a single cell, but you can make as many additional cells as you need. 
* Follow the instructions on the corresponding assignment Trello card for submitting your assignment. 

In [1]:
import pandas as pd 

data = pd.read_csv('pnp-train.txt',delimiter='\t',encoding='latin-1', # utf8 encoding didn't work for this
                  names=['type','name']) # supply the column names for the dataframe

# this next line creates a new column with the lower-cased first word
data['first_word'] = data['name'].map(lambda x: x.lower().split()[0])

In [2]:
data[:10]

,type,name,first_word
0,drug,Dilotab,dilotab
1,movie,Beastie Boys: Live in Glasgow,beastie
2,person,Michelle Ford-Eriksson,michelle
3,place,Ramsbury,ramsbury
4,place,Market Bosworth,market
5,drug,Cyanide Antidote Package,cyanide
6,person,Bill Johnson,bill
7,place,Ettalong,ettalong
8,movie,The Suicide Club,the
9,place,Pézenas,pézenas


In [3]:
data.describe()

,type,name,first_word
count,21001,21001,21001
unique,5,20992,13703
top,movie,Valentine,the
freq,6262,2,635


## 1. Write a probability function/distribution $P(T)$ over the types. 

Hints:

* The Counter library might be useful: `from collections import Counter`
* Write a function `def P(T='')` that returns the probability of the specific value for T
* You can access the types from the dataframe by calling `data['type']`

In [45]:
all_types = {'movie':6262, 'drug':5030, 'person':3836, 'place':3389, 'company':2484}
def P(T=''):
    return all_types[T]/sum(all_types.values())

## 2. What is `P(T='movie')` ?

In [46]:
P(T='movie')

0.29817627732012764

## 3. Show that your probability distribution sums to one.

In [47]:
P(T='movie') + P(T='drug') + P(T='person') + P(T='place') + P(T='company')

1.0

## 4. Write a joint distribution using the type and the first word of the name

Hints:

* The function is $P2(T,W_1)$
* You will need to count up types AND the first words, for example: ('person','bill)
* Using the [itertools.product](https://docs.python.org/2/library/itertools.html#itertools.product) function was useful for me here

In [48]:
#from collections import Counter
#print(data['first_word'].value_counts())
 
possibilities = []
for x,y in zip(data['type'], data['first_word']):
    possibilities.append([x,y])
    
def P2(T='',W1=''):
    return possibilities.count([T,W1])/len(possibilities)

## 5. What is P2(T='person', W1='bill')? What about P2(T='movie',W1='the')?

In [49]:
P2(T='person', W1='bill')

0.00047616780153326033

In [50]:
P2(T='movie', W1='the')

0.02747488214846912

## 6. Show that your probability distribution P(T,W1) sums to one.

In [51]:
round(sum([P2(T=a,W1=b) for a,b in set(zip(data['type'],data['first_word']))]),2)

1.0

## 7. Make a new function Q(T) from marginalizing over P(T,W1) and make sure that Q(T) sums to one.

Hints:

* Your Q function will call P(T,W1)
* Your check for the sum to one should be the same answer as Question 3, only it calls Q instead of P.

In [52]:
def Q(T=''):
    p = [P2(T,w) for w in set(data['first_word'])]
    return sum(p)

In [53]:
Q(T='movie')

0.2981762773201288

In [54]:
import math
math.floor(Q(T='movie') + Q(T='drug') + Q(T='person') + Q(T='company') + Q(T='place')) #result was 1.00000000000000034 before

1

## 8. What is the KL Divergence of your Q function and your P function for Question 1?

* Even if you know the answer, you still need to write code that computes it.

In [55]:
import math
(P(T='movie') * math.log(P('movie') / Q(T='movie')) + P(T='company') * math.log(P('company') / Q(T='company')) + P(T='place') * math.log(P('place') / Q(T='place')) + P(T='person') * math.log(P('person') / Q(T='person')) + P(T='drug') * math.log(P('drug') / Q(T='drug')))

-3.0083771608634536e-14

## 9. Convert from P(T,W1) to P(W1|T) 

Hints:

* Just write a comment cell, no code this time. 
* Note that $P(T,W1) = P(W1,T)$

#### Answer:
$P(T,W1) = P(T)*P(W1|T)$ =======> $P(W1|T) = P(T,W1)/(P(T)$

## 10. Write a function `Pwt` (that calls the functions you already have) to compute $P(W_1|T)$.

* This will be something like the multiplication rule, but you may need to change something

In [56]:
def Pwt(W1='', T=''):
    return P2(T, W1)/P(T)

## 11. What is P(W1='the'|T='movie')?

In [57]:
Pwt(W1='the',T='movie')

0.09214308527626956

## 12. Use Baye's rule to convert from P(W1|T) to P(T|W1). Write a function Ptw to reflect this. 

Hints:

* Call your other functions.
* You may need to write a function for P(W1) and you may need a new counter for `data['first_word']`

In [58]:
#P(T|W1) = ( P(W1|T)*P(T) ) / P(W1)
def Pw1(W1=''):
    L = list(data['first_word'])
    return L.count(W1)/len(L)

def Ptw(T='',W1=''):
    return (Pwt(W1,T)*P(T)) / Pw1(W1)


## 13 
### What is P(T='movie'|W1='the')? 
### What about P(T='person'|W1='the')?
### What about P(T='drug'|W1='the')?
### What about P(T='place'|W1='the')
### What about P(T='company'|W1='the')

In [59]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [60]:
Ptw(T='person',W1='the')

0.0

In [61]:
Ptw(T='drug',W1='the')

0.0

In [62]:
Ptw(T='place',W1='the')

0.0015748031496062992

In [63]:
Ptw(T='company',W1='the')

0.08976377952755905

## 14 Given this, if the word 'the' is found in a name, what is the most likely type?

**Answer:** Movie

## 15. Is Ptw(T='movie'|W1='the') the same as Pwt(W1='the'|T='movie') the same? Why or why not?

In [64]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [65]:
Pwt(W1='the', T='movie')

0.09214308527626956

#### Answer:
No, because the first one asks "what is the probability of the type being 'movie' given that the first word is 'the'? " while the second one asks "what is the probability of the first word being 'the' given that the type is 'movie'? "

## 16. Do you think modeling Ptw(T|W1) would be better with a continuous function like a Gaussian? Why or why not?

- Answer in a markdown cell


#### Answer:  
No, because we are dealing with discrete data (words) while countinous data are things that belong on a number range.

In [66]:
# from itertools import product
# l1 = ['greg', 'thomas', 'greg']
# l2 = [10, 20, 10]
# p = product(l1,l2, repeat=1)
# #print(type(p))

# A = ['g','b','t']
# B = ['r', 'p', 'y']
# qw = (print(x,y) for x in A for y in B)
#print(*qw)

In [44]:
from client.api.notebook import Notebook
ok = Notebook('a2.ok')
ok.auth(inline=True, force=True)

Assignment: A2 Python and Jupyter
OK, version v1.13.11


Open the following URL:

https://okpy.org/client/login/

After logging in, copy the code from the web page and paste it into the box.
Then press the "Enter" key on your keyboard.

Paste your code here: 5dkn6r6NKuu9KVMbNNlp3eRzHUs1De
Successfully logged in as emanuelhernandez@u.boisestate.edu


In [67]:
ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'A2-probability-information-theory.ipynb'.
Submit... 100% complete
Submission successful for user: emanuelhernandez@u.boisestate.edu
URL: https://okpy.org/boisestate/cs4-533/sp19/a2/submissions/gLBlKr

